# 通过时间反向传播
在Implemeting RNN from Scratch.ipynb 中，如果去掉梯度截断函数，我们会发现训练过程十分不稳定，梯度截断对于确保模型收敛至关重要。

为了更好地理解这个问题，需要对整个模型的计算过程进行回顾，在RNN中仍然是使用链式求导法则来计算梯度。


## RNN的梯度分析
假设在时间步$t$我们有：

- 隐状态 $h_t$
- 输入$x_t$
- 输出 $o_t$

隐变量可以拼接之后与隐藏层中的权重变量相乘，因此使用$w_h, w_o$来表示隐藏层和输出层的权重，每个时间步的隐状态和输出可以用如下公式表示：
$$
\begin{split}\begin{aligned}
&h_t = f(x_t,h_{t-1},w_h),\\
&o_t = g(h_t,w_o),\\
&\text{其中}w_h,w_o\text{为权重}\\
\end{aligned}\end{split}
$$

因此，我们有一个链:
$$\{\dots,(x_{t-1},h_{t-1},o_{t-1}),(x_{t},h_{t},o_{t}),\dots\}$$
他们通过循环计算彼此依赖，前向传播很简单，一次一个时间步遍历三元组$(x_{t},h_{t},o_{t})$，然后通过一个目标函数在所有$T$个时间步内评估输出$o_t$和标签$y_t$之间的差异：

$$
\mathcal{L}(x_1,\dots,x_T,y_1,\dots,y_T) = \frac{1}{T} \sum_{t=1}^T l(y_t,o_t)
$$

但是对于反向传播来说，我们计算$ \mathcal{L}$关于参数$ w_h$的梯度的时候，有：
$$
\begin{split}\begin{aligned}
\frac {\partial{L}}{\partial w_h} &= \frac {1}{T} \sum_{t=1}^T  \frac{\partial l(y_t,o_t)}{\partial w_h}\\
&= \frac {1}{T} \sum_{t=1}^T \frac{\partial l(y_t,o_t)}{\partial o_t} \frac{\partial g(h_t,w_o)}{\partial h_t}  \frac{\partial h_t}{\partial w_h} \\
\end{aligned}\end{split}
$$

上面乘积中第一项第二项很好计算，但是第三项 $\frac{\partial h_t}{\partial w_h} $ 的计算很麻烦，因为我们需要循环地计算参数$w_h$对$h_t$的影响，根据RNN中的递归运算得知，$h_t$的计算依赖于$h_{t-1}$又依赖于 $ {w_h}$其中$h_{t-1}$的计算也依赖于$w_k$，于是我们有：
$$
\begin{split}\begin{aligned}
h_t &= f(x_t,h_{t-1},w_h)\\
h_{t-1} &= f(x_{t-1},h_{t-2},w_h)\\
\frac {\partial h_t}{\partial w_h} &= \frac{\partial f(x_t,h_{t-1},w_h)}{\partial w_h} + \frac{\partial f(x_{t-1},h_{t-1},w_h)}{\partial h_{t-1}}\frac{\partial h_{t-1}}{\partial w_h}\\
\end{aligned}\end{split}
$$

其中，$h_{t-2} = f(x_t,h_{t-3},w_h),\dots,$一直迭代下去，我们有：
$$
\frac{\partial h_t}{\partial w_h} = \frac{\partial f(x_t,h_{t-1},w_h)}{\partial w_h} + \sum_{i=1}^{t-1}\left(\prod_{j=i+1}^t \frac {\partial f(x_j,h_{j-1},w_h)}{\partial h_{j-1}}\right) \frac{\partial f(x_i,h_{i-1},w_h)}{\partial w_h}
$$

当$t$很大的时候，这个链会非常长，需要想办法来处理这个问题

## 如何计算梯度？
### 1.完全计算
不稳定，不用！

### 2.截断时间步
在$\tau$时间步之后直接截断上面的链式求和计算，调用torch.Tensor.detach()直接从计算图中分离。这样算出来就是梯度的近似，在实践中这种方法的效果十分好，被称为通过时间反向传播，这样导致模型主要侧重于短期影响，而不是长期影响，在现实中，这种方法是可取的，因为它将估计值偏向更简单的结局。

### 3.随机截断
可以用一个随机变量来替换 $ \frac {\partial h_t}{\partial w_h}$，这个随机变量在预期中是正确的，但是会截断序列。这个随机变量是通过使用序列$ \xi_t$实现的，序列预定义了$ 0 \leq \pi_t \leq 1$,其中 $P(\xi_t =0) = 1-\pi, P(\xi_t = \pi_t^{-1} = \pi_t)$,因此 $E[\xi_t]=1$,用它来替换 $ \frac {\partial h_t}{\partial w_h}$得到：
$$z_t = \frac{\partial f(x_t,h_{t-1},w_h)}{\partial w_h} + \xi_t \frac{\partial f(x_{t-1},h_{t-1},w_h)}{\partial h_{t-1}}\frac{\partial h_{t-1}}{\partial w_h}$$

当$\xi_t =0 $的时候，算法终止在$t$这个时间步，这导致了不同长度序列的加权和。

遗憾的是，虽然随机截断在理论上更加具有吸引力，但是很可能由于多种因素在实践中不如常规的截断策略。

